In [ ]:
from collections import deque

In [ ]:
fn = 'day18.txt'

In [ ]:
def load_grid(fn):
    grid = {}
    
    with open(fn) as fh:
        for row, line in enumerate(fh):
            for col, c in enumerate(line.strip()):
                grid[complex(col, row)] = c
                if c == '@':
                    entrance = complex(col, row)

    return grid, entrance

In [ ]:
import sys

In [ ]:
def draw_grid(grid):
    max_x = max(int(p.real) for p in grid)
    max_y = max(int(p.imag) for p in grid)
    
    for row in range(max_y + 1):
        for col in range(max_x + 1):
            sys.stdout.write(grid[complex(col, row)])
        sys.stdout.write('\n')

In [ ]:
def distances_from(start, grid, keys_available):
    distances = {start: 0}
    to_check = deque()

    to_check.append(start)

    while len(to_check) > 0:
        position = to_check.popleft()
        for direction in [1, -1, 1j, -1j]:
            next_position = position + direction
            next_distance = distances[position] + 1
            next_char = grid[next_position]
            
            if next_char != '#' and ((not next_char.isupper()) or next_char in keys_available) and next_position not in distances:
                distances[next_position] = next_distance
                to_check.append(next_position)
    
    return distances

In [ ]:
def get_options(current_position, grid, keys_available):
    accessible = distances_from(current_position, grid, keys_available)
    options = {}
    for position in accessible:
        if position != current_position and grid[position].islower() and grid[position].upper() not in keys_available:
            options[grid[position]] = (position, accessible[position])
    return options

In [ ]:
def shortest_path(grid, current_position, keys_available, progress_bar, distance_so_far, best_so_far):
    progress_bar.update()
    options = get_options(current_position, grid, keys_available)

    if len(options) == 0:
        return distance_so_far
    
    possibilities = []
    
    for next_key, (next_position, distance_to) in options.items():
        new_keys = keys_available.copy()
        new_keys.add(next_key.upper())
        next_distance_so_far = distance_so_far + distance_to
        
        so_far_hash = (next_position, tuple(sorted(new_keys)))
        if next_distance_so_far < best_so_far.get(so_far_hash, np.inf):
            best_so_far[so_far_hash] = next_distance_so_far
            total_distance = shortest_path(grid, next_position, new_keys, p, next_distance_so_far, best_so_far)
            possibilities.append(total_distance)
            
    if len(possibilities) == 0:
        return np.inf
    else:
        return min(possibilities)

In [ ]:
fn = 'day18.txt'

In [ ]:
grid, entrance = load_grid(fn)
p = progress()
best_so_far = {}
shortest_path(grid, entrance, set(), p, 0, best_so_far)

In [ ]:
def load_grid_part2(fn):
    grid = {}
    entrances = []
    with open(fn) as fh:
        for row, line in enumerate(fh):
            for col, c in enumerate(line.strip()):
                grid[complex(col, row)] = c
                if c == '@':
                    entrances.append(complex(col, row))

    return grid, entrances

In [ ]:
def partition_grid(fn):
    grid, entrance = load_grid(fn)
    entrances = []
    grid[entrance] = '#'
    for d in [1, -1, 1j, -1j]:
        grid[entrance + d] = '#'
    for d in [(1 + 1j), (1 - 1j), (-1 + 1j), (-1 - 1j)]:
        grid[entrance + d] = '@'
        entrances.append(entrance + d)
    return grid, entrances

In [ ]:
grid, entrances = partition_grid('day18.txt')

In [ ]:
def get_options_multiple(current_positions, grid, keys_available):
    options = {}
    
    for robot, current_position in enumerate(current_positions):
        accessible = distances_from(current_position, grid, keys_available)

        for position in accessible:
            if position != current_position and grid[position].islower() and grid[position].upper() not in keys_available:
                options[grid[position]] = (robot, position, accessible[position])
    
    return options

In [ ]:
def shortest_path_multiple(grid, current_positions, keys_available, progress_bar, distance_so_far, best_so_far):
    progress_bar.update()
    options = get_options_multiple(current_positions, grid, keys_available)

    if len(options) == 0:
        return distance_so_far
    
    possibilities = []
    
    for next_key, (robot, next_position, distance_to) in options.items():
        new_keys = keys_available.copy()
        new_keys.add(next_key.upper())
        next_distance_so_far = distance_so_far + distance_to
        
        next_positions = current_positions.copy()
        next_positions[robot] = next_position
        
        so_far_hash = (tuple(next_positions), tuple(sorted(new_keys)))
        if next_distance_so_far < best_so_far.get(so_far_hash, np.inf):
            best_so_far[so_far_hash] = next_distance_so_far
        
            total_distance = shortest_path_multiple(grid, next_positions, new_keys, progress_bar, next_distance_so_far, best_so_far)
            possibilities.append(total_distance)
            
    if len(possibilities) == 0:
        return np.inf
    else:
        return min(possibilities)

In [ ]:
grid, entrances = load_grid_part2('day18_part2_test2.txt')

p = progress()
shortest_path_multiple(grid, entrances, set(), p, 0, {})

In [ ]:
grid, entrances = partition_grid('day18.txt')

p = progress()
shortest_path_multiple(grid, entrances, set(), p, 0, {})